In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
df = pd.ExcelFile("Business.xlsx")
df.sheet_names

import pandas as pd

Saving Business.xlsx to Business (2).xlsx


In [ ]:
# Read table properly with correct header row
# Table 2 (Nominal Value Added)
table2 = pd.read_excel("Business.xlsx", sheet_name="Table 2", skiprows=5)

# Drop the first column (the numbering column)
table2 = table2.drop(columns=table2.columns[0])

# Rename first column properly to "Industry"
table2 = table2.rename(columns={table2.columns[0]: "Industry"})

# Melt into long format
table2_long = table2.melt(id_vars="Industry",
                          var_name="Year",
                          value_name="Nominal_Value")

display(table2_long.head(20))





,Industry,Year,Nominal_Value
0,Space economy1,2012,110992
1,Private industries,2012,90312
2,"Agriculture, forestry, fishing, and hu...",2012,…
3,Farms,2012,…
4,"Forestry, fishing, and related act...",2012,…
5,Mining,2012,50
6,Oil and gas extraction,2012,37
7,"Mining, except oil and gas",2012,0
8,Support activities for mining,2012,13
9,Utilities,2012,1


In [ ]:
# Table 3 (Price Index)
table3 = pd.read_excel("Business.xlsx", sheet_name="Table 3", skiprows=5)

# Drop the first column (the numbering column)
table3 = table3.drop(columns=table3.columns[0])

# Rename first column properly to "Industry"
table3 = table3.rename(columns={table3.columns[0]: "Industry"})

# Melt into long format
table3_long = table3.melt(id_vars="Industry",
                          var_name="Year",
                          value_name="Price_Index")

display(table3_long.head(20))


,Industry,Year,Price_Index
0,Space economy1,2012,108.853
1,Private industries,2012,113.087
2,"Agriculture, forestry, fishing, and hu...",2012,…
3,Farms,2012,…
4,"Forestry, fishing, and related act...",2012,…
5,Mining,2012,94.206
6,Oil and gas extraction,2012,94.219
7,"Mining, except oil and gas",2012,99.838
8,Support activities for mining,2012,94.252
9,Utilities,2012,104.662


In [ ]:
# Merge and calculate real value
df = pd.merge(table2_long, table3_long, on=["Industry", "Year"], how="inner")

# Drop rows where either Nominal_Value or Price_Index is missing
df = df.dropna(subset=["Nominal_Value", "Price_Index"])

# Calculate Real Value
df["Real_Value"] = df["Nominal_Value"] * (100 / df["Price_Index"])

# Fix year column
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df.dropna(subset=["Year"])
df["Year"] = df["Year"].astype(int)

# Keep only 2012–2023
df = df[(df["Year"] >= 2012) & (df["Year"] <= 2023)]

display(df.head(20))




,Industry,Year,Nominal_Value,Price_Index,Real_Value
0,Space economy1,2012,110992.0,108.853,101965.035415
1,Private industries,2012,90312.0,113.087,79860.638270
5,Mining,2012,50.0,94.206,53.075176
6,Oil and gas extraction,2012,37.0,94.219,39.270211
7,"Mining, except oil and gas",2012,0.0,99.838,0.000000
8,Support activities for mining,2012,13.0,94.252,13.792811
9,Utilities,2012,1.0,104.662,0.955457
10,Construction,2012,229.0,86.315,265.307305
11,Manufacturing,2012,34461.0,142.443,24192.835029
12,Durable goods,2012,34033.0,142.941,23809.124044


In [ ]:
# Pivot so years are across columns, industries are rows
df_wide = df.pivot_table(index="Industry",
                         columns="Year",
                         values="Real_Value")  # or Nominal_Value / Price_Index

# Reset index to keep Industry as a column
df_wide = df_wide.reset_index()

display(df_wide.head())

Year,Industry,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Housing,0.000000,0.000000,0.000000,0.000000,1.014086,1.0,0.983719,0.977670,0.952862,0.940663,0.923788,0.909248
1,Other real estate,0.000000,0.000000,1.012340,1.004248,2.036598,3.0,2.936110,2.910135,4.703669,3.694433,3.616342,3.555777
2,Accommodation,3.015621,4.006290,3.988036,3.934878,2.961647,3.0,3.059757,3.074810,2.066735,2.087813,3.143797,5.254309
3,Administrative and support ser...,20.314262,38.796888,57.642716,76.788831,97.739768,109.0,98.607775,100.994406,97.090124,105.399633,129.632524,144.969556
4,Ambulatory health care services,1.925224,1.944504,1.956698,1.969105,3.998920,4.0,3.001231,4.022526,3.020966,4.083008,4.071454,4.042487


In [ ]:
df.to_excel("cleaned_space_economy.xlsx", index=False)

In [ ]:
df_wide = df.pivot_table(index="Industry",
                         columns="Year",
                         values="Real_Value")  # or "Nominal_Value", "Price_Index"

# Reset index so Industry becomes a column again
df_wide = df_wide.reset_index()

# Save to Excel
df_wide.to_excel("cleaned_space_economy_wide.xlsx", index=False)